In [2]:
import os 

# os.chdir(os.path.abspath(os.path.pardir))
os.chdir("/home/adpakw/face_recognition_system")

In [3]:
import torch
print(torch.version.cuda)
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")
print(f"Current GPU: {torch.cuda.current_device()}")
print(f"GPU name: {torch.cuda.get_device_name(0)}")

12.4
PyTorch version: 2.6.0+cu124
CUDA available: True
GPU count: 1
Current GPU: 0
GPU name: NVIDIA GeForce RTX 3060


In [ ]:
from app.clients.postgres import PostgreSQLClient

pg = PostgreSQLClient()
pg.connect()
pg.execute_query(query="SELECT table_name FROM information_schema.tables", return_df=True)

Успешное подключение к PostgreSQL


,table_name
0,pg_statistic
1,pg_type
2,pg_foreign_table
3,pg_authid
4,pg_shadow
...,...
198,foreign_servers
199,_pg_foreign_tables
200,user_mapping_options
201,foreign_tables


In [5]:
from app.clients.postgres import PostgreSQLClient

pg = PostgreSQLClient()
pg.connect()
pg.execute_query(query="DROP TABLE videos")

Успешное подключение к PostgreSQL


In [66]:
from app.clients.postgres import PostgreSQLClient

pg = PostgreSQLClient()
pg.connect()
pg.execute_query(query="SELECT * FROM video_analysis_results", return_df=True)

Успешное подключение к PostgreSQL


,video_name,frame_number,timestamp,person_bbox_x1,person_bbox_x2,person_bbox_y1,person_bbox_y2,person_detection_conf,face_bbox_x1,face_bbox_x2,face_bbox_y1,face_bbox_y2,face_detection_conf,person_name,person_identification_conf
0,parsa.mp4,1,0.000000,1648,1918,843,1078,0.852828,1750.0,1816.0,861.0,968.0,0.830725,unknown,0.000000
1,parsa.mp4,1,0.000000,236,497,438,771,0.803308,361.0,432.0,447.0,542.0,0.933895,unknown,0.000000
2,parsa.mp4,2,0.033333,1648,1918,843,1078,0.850486,1750.0,1816.0,861.0,968.0,0.831083,unknown,0.000000
3,parsa.mp4,2,0.033333,236,497,438,771,0.805214,361.0,432.0,447.0,542.0,0.933912,unknown,0.000000
4,parsa.mp4,3,0.066667,1648,1918,843,1078,0.847480,1752.0,1812.0,866.0,967.0,0.844701,unknown,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,parsa.mp4,146,4.833333,1650,1915,843,1077,0.847509,1732.0,1803.0,856.0,963.0,0.859394,unknown,0.000000
325,parsa.mp4,146,4.833333,299,517,497,840,0.813045,401.0,474.0,518.0,617.0,0.911686,palmasizade,0.705086
326,parsa.mp4,147,4.866667,1650,1915,843,1077,0.847180,1732.0,1803.0,856.0,963.0,0.858718,unknown,0.000000
327,parsa.mp4,147,4.866667,299,517,497,840,0.814926,397.0,474.0,518.0,618.0,0.911368,palmasizade,0.000000


In [67]:
from app.clients.postgres import PostgreSQLClient

pg = PostgreSQLClient()
pg.connect()
pg.execute_query(query="TRUNCATE video_analysis_results", return_df=True)

Успешное подключение к PostgreSQL
